In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make sure the graph is refresh
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)


In [4]:
# one_hot encoding 5, 6, 7, 8, 9 for all labels
def one_hot_encoding(y):
    tmp_y = np.zeros([y.shape[0], 5])
    for i in range(y.shape[0]):
        tmp_y[i][y[i]] = 1
    return tmp_y

y_train2 = one_hot_encoding(y_train2)
y_valid2 = one_hot_encoding(y_valid2)
y_test2 = one_hot_encoding(y_test2)

In [5]:
def train_with_only_softmax(X_train, y_train, X_validate, y_validate, train_op, epoch_bound, stop_threshold, batch_size, testing=False, new_saver=None, new_model_path=None):
    
    early_stop = 0
    winner_loss = np.infty
    winner_accuracy = 0.0
    
    t0 = time.time()
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training,:]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(train_op, feed_dict={X: X_train[batch*batch_size:], y: y_train[batch*batch_size:], mode:'TRAIN'})
            else:
                sess.run(train_op, feed_dict={X: X_train[batch*batch_size : (batch+1)*batch_size], y: y_train[batch*batch_size : (batch+1)*batch_size], mode:'TRAIN'})

        # compute validation accuracy
        cur_accuracy, cur_loss = evaluate_with_only_softmax(X_validate, y_validate)

        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_loss > cur_loss:
            early_stop = 0
            winner_loss = cur_loss
            winner_accuracy = cur_accuracy
            # save best model in testing phase
            if testing == True:
                save_path = new_saver.save(sess, new_model_path + ".ckpt")
        else:
            early_stop += 1
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(epoch, cur_loss, winner_loss, cur_accuracy * 100))
        if early_stop == stop_threshold:
            print("Early Stop.")
            break
    t1 = time.time()
    print("Total training time of HW3-1: {:.1f}s".format(t1 - t0))
    
    return winner_accuracy, winner_loss

# evaluate model: compute accuracy, precision, recall
def evaluate_with_only_softmax(Inputs, Labels):
    global Y_probability, loss
    y_predict = sess.run(Y_probability, feed_dict={X: Inputs, y: Labels, mode:'EVAL'})
    correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(Labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    acc = sess.run(accuracy, feed_dict={X: Inputs, y:Labels, mode:'EVAL'})
    loss_val = sess.run(loss, feed_dict={X: Inputs, y:Labels, mode:'EVAL'})        
    return acc, loss_val

In [6]:
# 3-1: Softmax Only
reset_graph()
pretrained_model_path = "./saved_model/Team35_HW2"
new_model_path = "./saved_model/Team35_HW3_1"
pretrained_saver = tf.train.import_meta_graph(pretrained_model_path + ".ckpt.meta")
new_saver = tf.train.Saver()

In [7]:
# define hyper-parameters
learning_rate = 0.01
batch_size = 64
epoch_bound = 1000
stop_threshold = 20

# 取得要transfer的圖 get graph for 1~5 layers (transfer layers) 
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
mode = tf.get_default_graph().get_tensor_by_name("mode:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_probability = tf.get_default_graph().get_tensor_by_name("Y_probability:0")
logits = Y_probability.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

# create new training layers
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")

# define new training steps
optimizer = tf.train.AdamOptimizer(learning_rate, name="AdamOp_3-1")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    
    # init weights
    sess.run(init)
    
    # 取得tenfor的數值 restore value of transfer layers
    pretrained_saver.restore = (sess, pretrained_model_path + ".ckpt")

    # 初始新的layer initialize value for softmax layer
    for var in output_layer_vars:
        sess.run(var.initializer)
    
    winner_accuracy, winner_loss = train_with_only_softmax(X_train2, y_train2, X_valid2, y_valid2, training_op, epoch_bound, stop_threshold, batch_size, testing=True, new_saver=new_saver, new_model_path=new_model_path)

    new_saver.restore(sess, new_model_path + ".ckpt")
    test_accuracy, test_loss = evaluate_with_only_softmax(X_test2, y_test2)
    print("Test accuracy: {:.2f}%".format(test_accuracy * 100))

0	Validation loss: 192.551804	Best loss: 192.551804	Accuracy: 46.00%
1	Validation loss: 153.083832	Best loss: 153.083832	Accuracy: 60.67%
2	Validation loss: 132.998337	Best loss: 132.998337	Accuracy: 63.33%
3	Validation loss: 119.425430	Best loss: 119.425430	Accuracy: 73.33%
4	Validation loss: 110.371307	Best loss: 110.371307	Accuracy: 74.00%
5	Validation loss: 106.274345	Best loss: 106.274345	Accuracy: 75.33%
6	Validation loss: 101.357330	Best loss: 101.357330	Accuracy: 76.67%
7	Validation loss: 99.859924	Best loss: 99.859924	Accuracy: 75.33%
8	Validation loss: 96.798958	Best loss: 96.798958	Accuracy: 78.67%
9	Validation loss: 95.876167	Best loss: 95.876167	Accuracy: 76.67%
10	Validation loss: 93.794830	Best loss: 93.794830	Accuracy: 80.67%
11	Validation loss: 92.263268	Best loss: 92.263268	Accuracy: 78.67%
12	Validation loss: 91.889305	Best loss: 91.889305	Accuracy: 80.00%
13	Validation loss: 91.156952	Best loss: 91.156952	Accuracy: 78.67%
14	Validation loss: 89.887634	Best loss: 89.

In [9]:
def train_with_softmax_and_cache_h5(X_train, y_train, X_validate, y_validate, train_op, epoch_bound, stop_threshold, batch_size, testing=False, new_saver=None, new_model_path=None):
    
    early_stop = 0
    winner_loss = np.infty
    winner_accuracy = 0.0
    
    t0 = time.time()
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training,:]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(train_op, feed_dict={h5_out: X_train[batch*batch_size:], y: y_train[batch*batch_size:], mode:'TRAIN'})
            else:
                sess.run(train_op, feed_dict={h5_out: X_train[batch*batch_size : (batch+1)*batch_size], y: y_train[batch*batch_size : (batch+1)*batch_size], mode:'TRAIN'})

        # compute validation accuracy
        cur_accuracy, cur_loss = evaluate_with_softmax_and_cache_h5(X_validate, y_validate)

        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_loss > cur_loss:
            early_stop = 0
            winner_loss = cur_loss
            winner_accuracy = cur_accuracy
            # save best model in testing phase
            if testing == True:
                save_path = new_saver.save(sess, new_model_path + ".ckpt")
        else:
            early_stop += 1
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(epoch, cur_loss, winner_loss, cur_accuracy * 100))
        if early_stop == stop_threshold:
            print("Early Stop.")
            break
    t1 = time.time()
    print("Total training time of HW3-1: {:.1f}s".format(t1 - t0))
    
    return winner_accuracy, winner_loss

# evaluate model: compute accuracy, precision, recall
def evaluate_with_softmax_and_cache_h5(Inputs, Labels):
    global Y_probability, loss
    y_predict = sess.run(Y_probability, feed_dict={h5_out: Inputs, y: Labels, mode:'EVAL'})
    correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(Labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    acc = sess.run(accuracy, feed_dict={h5_out: Inputs, y:Labels, mode:'EVAL'})
    loss_val = sess.run(loss, feed_dict={h5_out: Inputs, y:Labels, mode:'EVAL'})        
    return acc, loss_val

In [10]:
# use the function, tf.get_default_graph().get_operations(), to find right tensor name
#print(tf.get_default_graph().get_operations())
h5_dropout = tf.get_default_graph().get_tensor_by_name("dropout5/Identity:0")
h5_out = h5_dropout.op.inputs[0]

new_model_path = "./saved_model/Team35_HW3_2"
new_saver = tf.train.Saver()

In [13]:
with tf.Session() as sess:
    
    # init weights
    sess.run(init)
    
    # 取得tenfor的數值 restore value of transfer layers
    pretrained_saver.restore = (sess, pretrained_model_path + ".ckpt")

    # 初始新的layer initialize value for softmax layer
    for var in output_layer_vars:
        sess.run(var.initializer)
    
    h5_train = sess.run(h5_out, feed_dict={X: X_train2, y: y_train2, mode:'TRAIN'})
    h5_valid = sess.run(h5_out, feed_dict={X: X_valid2, y: y_valid2, mode:'TRAIN'})
    
    # training phase
    winner_accuracy, winner_loss = train_with_softmax_and_cache_h5(h5_train, y_train2, h5_valid, y_valid2, training_op, epoch_bound, stop_threshold, batch_size, testing=True, new_saver=new_saver, new_model_path=new_model_path)

    # testing phase
    new_saver.restore(sess, new_model_path + ".ckpt")
    test_accuracy, test_loss = evaluate_with_only_softmax(X_test2, y_test2)
    print("Test accuracy: {:.2f}%".format(test_accuracy * 100))

0	Validation loss: 186.019379	Best loss: 186.019379	Accuracy: 50.00%
1	Validation loss: 149.375626	Best loss: 149.375626	Accuracy: 61.33%
2	Validation loss: 129.953323	Best loss: 129.953323	Accuracy: 67.33%
3	Validation loss: 118.137283	Best loss: 118.137283	Accuracy: 72.00%
4	Validation loss: 110.139992	Best loss: 110.139992	Accuracy: 70.67%
5	Validation loss: 104.321075	Best loss: 104.321075	Accuracy: 74.00%
6	Validation loss: 100.331009	Best loss: 100.331009	Accuracy: 76.67%
7	Validation loss: 98.222519	Best loss: 98.222519	Accuracy: 77.33%
8	Validation loss: 97.048592	Best loss: 97.048592	Accuracy: 77.33%
9	Validation loss: 94.297844	Best loss: 94.297844	Accuracy: 79.33%
10	Validation loss: 93.406281	Best loss: 93.406281	Accuracy: 76.67%
11	Validation loss: 91.472527	Best loss: 91.472527	Accuracy: 78.67%
12	Validation loss: 90.616837	Best loss: 90.616837	Accuracy: 77.33%
13	Validation loss: 90.367195	Best loss: 90.367195	Accuracy: 78.00%
14	Validation loss: 89.149986	Best loss: 89.